In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("LinReg").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/13 14:33:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/02/13 14:33:17 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [6]:
from pyspark.ml.regression import LinearRegression

In [4]:
df = spark.read.format('libsvm').load("data/bodyfat.txt")
df.show()

23/02/13 14:46:37 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.
+------+--------------------+
| label|            features|
+------+--------------------+
|1.0708|(14,[0,1,2,3,4,5,...|
|1.0853|(14,[0,1,2,3,4,5,...|
|1.0414|(14,[0,1,2,3,4,5,...|
|1.0751|(14,[0,1,2,3,4,5,...|
| 1.034|(14,[0,1,2,3,4,5,...|
|1.0502|(14,[0,1,2,3,4,5,...|
|1.0549|(14,[0,1,2,3,4,5,...|
|1.0704|(14,[0,1,2,3,4,5,...|
|  1.09|(14,[0,1,2,3,4,5,...|
|1.0722|(14,[0,1,2,3,4,5,...|
| 1.083|(14,[0,1,2,3,4,5,...|
|1.0812|(14,[0,1,2,3,4,5,...|
|1.0513|(14,[0,1,2,3,4,5,...|
|1.0505|(14,[0,1,2,3,4,5,...|
|1.0484|(14,[0,1,2,3,4,5,...|
|1.0512|(14,[0,1,2,3,4,5,...|
|1.0333|(14,[0,1,2,3,4,5,...|
|1.0468|(14,[0,1,2,3,4,5,...|
|1.0622|(14,[0,1,2,3,4,5,...|
| 1.061|(14,[0,1,2,3,4,5,...|
+------+--------------------+
only showing top 20 rows



In [7]:
lr = LinearRegression(featuresCol='features', labelCol='label', predictionCol='prediction')
lr_model = lr.fit(df)

23/02/13 14:51:50 WARN Instrumentation: [6133500b] regParam is zero, which might cause numerical instability and overfitting.
23/02/13 14:51:50 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [8]:
lr_model.coefficients

DenseVector([-0.0022, 0.0, 0.0, -0.0, 0.0, 0.0001, -0.0002, 0.0001, -0.0001, -0.0, -0.0002, -0.0002, -0.0, 0.0004])

In [9]:
lr_model.intercept

1.0981463302283172

In [10]:
train, test = df.randomSplit([0.7, 0.3])

In [11]:
lr_model = lr.fit(train)

23/02/13 14:56:17 WARN Instrumentation: [c4a23d4d] regParam is zero, which might cause numerical instability and overfitting.


In [18]:
test_results = lr_model.evaluate(test)

In [13]:
test_results.rootMeanSquaredError

0.004955289095027164

In [15]:
unlabeled_data = test.select('features')

In [16]:
predictions = lr_model.transform(unlabeled_data)
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|(14,[0,1,2,3,4,5,...|  1.00826973518771|
|(14,[0,1,2,3,4,5,...| 1.020167641810168|
|(14,[0,1,2,3,4,5,...|1.0253768505578704|
|(14,[0,1,2,3,4,5,...|1.0265532108129278|
|(14,[0,1,2,3,4,5,...| 1.027210884332204|
|(14,[0,1,2,3,4,5,...|1.0270229682287173|
|(14,[0,1,2,3,4,5,...|1.0285355399346798|
|(14,[0,1,2,3,4,5,...|1.0302070498329485|
|(14,[0,1,2,3,4,5,...|1.0314249809154912|
|(14,[0,1,2,3,4,5,...|1.0313546073923379|
|(14,[0,1,2,3,4,5,...|1.0370764500660496|
|(14,[0,1,2,3,4,5,...|1.0385204709025855|
|(14,[0,1,2,3,4,5,...|1.0390197117424678|
|(14,[0,1,2,3,4,5,...|1.0402402882397581|
|(14,[0,1,2,3,4,5,...|1.0416181633948498|
|(14,[0,1,2,3,4,5,...|1.0427221778036426|
|(14,[0,1,2,3,4,5,...|1.0442142852870713|
|(14,[0,1,2,3,4,5,...|1.0440840392938042|
|(14,[0,1,2,3,4,5,...|  1.04482570471748|
|(14,[0,1,2,3,4,5,...|1.0464583634207476|
+--------------------+------------

In [21]:
preds = lr_model.predict(predictions)

Py4JJavaError: An error occurred while calling o147.predict.
: java.lang.ClassCastException: org.apache.spark.sql.Dataset cannot be cast to org.apache.spark.ml.linalg.Vector
	at org.apache.spark.ml.regression.LinearRegressionModel.predict(LinearRegression.scala:691)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.ClientServerConnection.waitForCommands(ClientServerConnection.java:182)
	at py4j.ClientServerConnection.run(ClientServerConne